<a href="https://colab.research.google.com/github/danielsoy/ADer/blob/main/padim_train1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone https://github.com/danielsoy/anodet_29-08-2024.git

Cloning into 'anodet_29-08-2024'...
remote: Enumerating objects: 527, done.
remote: Counting objects: 100% (101/101), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 527 (delta 81), reused 71 (delta 68), pack-reused 426 (from 1)
Receiving objects: 100% (527/527), 5.88 MiB | 25.19 MiB/s, done.
Resolving deltas: 100% (293/293), done.


In [4]:
cd /content/anodet_29-08-2024/


/content/anodet_29-08-2024


In [6]:
pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 65.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 47.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 30.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

# First cell - imports and device setup
import os
import anodet
import numpy as np
import torch
import cv2
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt

# Check GPU availability
if torch.cuda.is_available():
    print("GPU is available")
    print("GPU Name:", torch.cuda.get_device_name(0))
    device = torch.device('cuda')
else:
    print("GPU is not available")
    device = torch.device('cpu')


# Initialize model and paths las cell
MODEL_DATA_PATH = os.path.realpath("./distributions/")
os.makedirs(MODEL_DATA_PATH, exist_ok=True)

# Initialize model on GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
padim = anodet.Padim(backbone='resnet18', device=device)

# Create a custom dataset that ensures tensors are on the correct device
class DeviceDataset(anodet.AnodetDataset):
    def __getitem__(self, index):
        img, label, mask = super().__getitem__(index)
        # Convert to tensor if not already and move to GPU
        img = torch.tensor(img, device=device) if not torch.is_tensor(img) else img.to(device)
        label = torch.tensor(label, device=device) if not torch.is_tensor(label) else label.to(device)
        mask = torch.tensor(mask, device=device) if not torch.is_tensor(mask) else mask.to(device)
        return img, label, mask

# Load your dataset with the custom class
dataset = DeviceDataset(os.path.join("C:\\anodet/notebooks/video_pro\working folder/nueva\datasets\DATASET B"))
dataloader = DataLoader(dataset, batch_size=1)
print("Number of images in dataset:", len(dataloader.dataset))

# Ensure the model's internal components are on GPU
padim.to_device(device)

# Train the model
print("Training model...")
try:
    padim.fit(dataloader, extractions=1)
    print("Training completed successfully")
except RuntimeError as e:
    print("Error during training:", str(e))
    raise

# Verify training was successful
print("\nVerifying model attributes after training:")
print("Mean shape:", padim.mean.shape if padim.mean is not None else "None")
print("Mean device:", padim.mean.device if padim.mean is not None else "None")
print("Cov_inv shape:", padim.cov_inv.shape if padim.cov_inv is not None else "None")
print("Cov_inv device:", padim.cov_inv.device if padim.cov_inv is not None else "None")

# Only save if training was successful
if padim.mean is not None and padim.cov_inv is not None:
    print("\nSaving model files...")
    torch.save(padim.mean, os.path.join(MODEL_DATA_PATH, "18_mean.pt"))
    torch.save(padim.cov_inv, os.path.join(MODEL_DATA_PATH, "18_cov_inv.pt"))
    print("Model saved successfully")
else:
    print("\nERROR: Training did not complete successfully")

<>:39: SyntaxWarning: invalid escape sequence '\w'
<>:39: SyntaxWarning: invalid escape sequence '\w'
C:\Users\libro\AppData\Local\Temp\ipykernel_16796\626273045.py:39: SyntaxWarning: invalid escape sequence '\w'
  dataset = DeviceDataset(os.path.join("C:\\anodet/notebooks/video_pro\working folder/nueva\datasets\DATASET B"))


GPU is available
GPU Name: NVIDIA GeForce GTX 1080 Ti
Number of images in dataset: 240
Training model...


Feature extraction: 100%|██████████| 240/240 [00:16<00:00, 14.19it/s]


Training completed successfully

Verifying model attributes after training:
Mean shape: torch.Size([3136, 100])
Mean device: cuda:0
Cov_inv shape: torch.Size([3136, 100, 100])
Cov_inv device: cuda:0

Saving model files...
Model saved successfully
